In [24]:
from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow as tf
import os
import vggish_input
import vggish_params
import vggish_slim
from pydub import AudioSegment
from audioModel import predict, train
from pydub import AudioSegment, generators
import IPython.display as ipd
from IPython.core.display import display, HTML
from audioInput import getLaughTracks, getNoise, processWavFile
from audioDisplay import predictAudio
import random
from audioUtils import downloadYtAndPrepareAudio, shell, readFolder
from audioTrain import trainAndSaveAndPredict, trainForNoise

def getGenerator(generator, seconds):    
    audio = AudioSegment.empty()
    for i in range(seconds):
        audio = audio + generator().to_audio_segment()
    return audio
        
def buildNoise(path, seconds):
    if os.path.isfile(path):    
        shell('rm %s' % path)
    SAMPLE_RATE = 44100
    
    def curriedSine():
        return generators.Sine(440)

    data = AudioSegment.empty()
    for i in range(seconds * 2):
        generator = random.choice([curriedSine, generators.WhiteNoise])
        data = data + getGenerator(generator, 1)
    #data = data + getGenerator(curriedSine, 3) + getGenerator(generators.WhiteNoise, 3)        
    
    audio = AudioSegment(data.get_array_of_samples(), sample_width=2, channels=2, frame_rate=44100)
    audio.export(path, format='wav')
    #display(ipd.Audio(path))    
    
def getModel(path):
    files = readFolder('model/%s' % path)
    if len(files) > 0:
        return '%s/%s' % (path, files[0])
    return None



In [25]:
buildNoise('data/noise_5.wav', 5)
audio = AudioSegment.from_file('data/noise_5.wav', format="wav").set_channels(1)
print(len(audio))

5000


In [19]:
preds, audio = predictAudio('data/noise_5.wav', 'noise_100', ['sine', 'const', 'noise'], number_of_classes=3) 
print(len(preds))
print(audio)

loading ./model/noise_100/20-20/model
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/noise_100/20-20/model


5
[[[ 6.81729691  8.17968842  8.79310926 ...  4.25599191  4.18060643
    4.66296442]
  [ 6.7490601   8.13943612  8.76854253 ...  3.47534768  4.18105932
    4.52777072]
  [ 6.62232967  8.06519436  8.7257061  ...  3.95311028  4.01013175
    4.48419575]
  ...
  [ 6.61723079  8.06618808  8.72516437 ...  4.01111436  4.17400471
    4.60457887]
  [ 6.74527709  8.13993887  8.76818235 ...  3.95292404  4.43291187
    4.74145909]
  [ 6.81240349  8.18006604  8.79348207 ...  4.28594891  4.398441
    4.81720383]]

 [[ 6.74865667  8.13919659  8.76880305 ...  3.60187865  4.32114935
    4.64371859]
  [ 6.62232967  8.06519436  8.7257061  ...  3.95311028  4.01013175
    4.48419574]
  [ 6.61723079  8.06618808  8.72516437 ...  4.01111436  4.17400471
    4.60457886]
  ...
  [ 6.75583692  8.13862855  8.76811374 ...  3.99662013  4.41779066
    4.73573525]
  [ 6.81729627  8.17968802  8.79310897 ...  4.25650065  4.18335971
    4.65173085]
  [ 6.74853788  8.13965399  8.76858842 ...  3.51159692  4.2072901
    4.5

In [20]:
audio.shape

(5, 96, 64)

In [3]:
predictAudio('data/noise.wav', getModel('noise_200'), number_of_classes = 3)